<a href="https://colab.research.google.com/github/guilhermefritz/SQL_Projects/blob/main/Transforma%C3%A7oes_dados_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## Feito por Guilherme Fritz







### Esse projeto tem o objetivo de demonstrar as transformações que são aplicadas constantemente em variaveis  para a realização de análises aprofundadas  e construção de gráficos .
#### * Categorização
#### * Enconding (Label Encoding e One -Hot enconding)
#### * Binarização

---------------------------------------------------------------------------------------------

Banco de dados -Avaliação de carros.

Disponivel em https://archive.ics.uci.edu/dataset/19/car+evaluation

------------------------------------------------------------------

### Dicionário dos dados


#### Variaveis                    Car acceptability
              
- buying -               Preço de compra em categorias ['vhigh','high','med','low'][Muito alto,alto,medio,baixo]

- maint  -              Preço de manutenção        

- doors  -             numero de portas

- persons -            capacidade de pessoas

- lug_boot -          tamanho do porta - malas ['small','med','big']

- safety   -          Nivel de segurança do carro['low','med','big']

- Target    -         Alvo ['unacceptable(unacc)', 'acceptable(acc)', 'good', 'very good(vhigh']=[Inaceitavel,aceitavel,bom,muito bom ]

In [35]:
!pip install -q ipython-sql
#Instalando a biblioteca que permite utilizar banco de dados com python

In [6]:
import sqlite3
import pandas as pd

In [7]:
df=pd.read_csv('D:\datasets\car_data_new.csv')

In [8]:
cnn = sqlite3.connect('D:/SQL/banco.db2')

In [37]:
#Celular que alimenta o banco sql com os dados,deixei comentado por que ja havia alimentado.
#df.to_sql('Car',cnn)

In [71]:
#Informaçoes das variaveis
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   buying   1728 non-null   object
 1   maint    1728 non-null   object
 2   doors    1728 non-null   object
 3   persons  1728 non-null   object
 4   lug_bot  1728 non-null   object
 5   safety   1728 non-null   object
 6   Target   1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [9]:
#Para poder utilizar a magica sql 
%load_ext sql

In [10]:
%sql sqlite:///D:/SQL/banco.db2

### Assim estava o banco de dados antes das transformações:

In [12]:
%%sql
SELECT * FROM Car
LIMIT 10
#Limit limita os dados no caso em 10 linhas

 * sqlite:///D:/SQL/banco.db2
Done.


index,buying,maint,doors,persons,lug_bot,safety,Target
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


-------------------------------------------------

 #### * Obs: Necessário antes de cada transformaçao a verificação de valores ausentes em cada variavel,pode-se fazer isso utilizando o DISTINCT
    

### !!!!!Cada mudança que aplico ,no final trago a coluna antiga comparada com a  coluna transformada

## -Categorização

   * O objetivo da categorização seria para diminuir a dimensionalidade da variavel categorica e facilitar analises

#### Verifiquei que a variavel categoerica 'buying' era composta por 4 classes :['vhigh' 'high' 'med' 'low'] e transformamos para que tivesse 2 classes :['Alto' ,'Baixo']

In [15]:
#Alem da magica sql trago a opçao de fazer os comandos pelo cursor 
conn = sqlite3.connect('D:/SQL/banco.db2')

# Criar um cursor
cursor = conn.cursor()

cursor.execute(''' SELECT  DISTINCT buying,
      CASE
        WHEN buying= 'vhigh' OR buying='high' THEN 'Alto'
        WHEN buying= 'med' OR buying='low' THEN 'Baixo'
      
      END as buying_less
    FROM Car ''')

resultados = cursor.fetchall()




# Exibir os resultados
df = pd.DataFrame(resultados, columns=['buying','buying_less'])
print(df)

  buying buying_less
0  vhigh        Alto
1   high        Alto
2    med       Baixo
3    low       Baixo


# -Enconding

* O objetivo do enconding seria transformar a variavel categorica para sua representação númerica

### Label Encoding

#### -Verifiquei que a variavel categorica 'safety' era composta por 3 classes :['low' 'med' 'high'],apliquei o enconding para obter a representação numerica da variavel que ficou : [1,2,3]

In [16]:


# Executar a consulta SQL
cursor.execute('''
   SELECT  DISTINCT safety,
      CASE
        WHEN safety='low' THEN 1
        WHEN safety='med' THEN 2
        WHEN safety='high' THEN 3
      END as safety_num
    FROM Car
''')


# Obter os resultados
resultados = cursor.fetchall()




# Exibir os resultados
df = pd.DataFrame(resultados, columns=['safety','safety_num'])
print(df)

  safety  safety_num
0    low           1
1    med           2
2   high           3


-------------------------------------------------------------------

#### Verifiquei que a variavel 'Target' era categorica e que era composta por 4 classes :['unacc' 'acc' 'vgood','good'],apliquei o enconding para obter a representação numerica da variavel que ficou assim [1,2,3,4]

In [17]:
cursor.execute(''' SELECT  DISTINCT Target,
      CASE
        WHEN Target= 'unacc' THEN 1
        WHEN Target= 'acc' THEN 2
        WHEN Target= 'vgood' THEN 3
        WHEN Target= 'good' THEN 4
        
      
      END as target_num
    FROM Car ''')

resultados = cursor.fetchall()

cursor.close()
conn.close()


# Exibir os resultados
df = pd.DataFrame(resultados, columns=['Target','target_num'])
print(df)

  Target  target_num
0  unacc           1
1    acc           2
2  vgood           3
3   good           4


###  One-hot-enconding

One-hot-enconding: Tecnica utilizada para transformar as variaveis categoricas em variaveis dummy

#### Verifiquei que a variavel 'lug_bot' estava categorica e que era composta por 3 classes :['small','med','big'] e representação numerica da variavel que ficou distribuida em tres colunas, em que quando a classe em questão está presente o valor é '1' e quando não está presente o valor é '0' na tabela , regra aplicada  em todas as  classes.

In [79]:
%%sql

SELECT DISTINCT lug_bot,
  CASE WHEN lug_bot='small' THEN 1 ELSE 0 END AS l_bot_small,
  CASE WHEN lug_bot='med' THEN 1 ELSE 0 END AS l_bot_med,
  CASE WHEN lug_bot='big' THEN 1 ELSE 0 END AS l_bot_big
FROM Car;







 * sqlite:///D:/SQL/banco.db2
   sqlite:///D:/SQL/banco.db3
   sqlite:///D:/SQL/banco.db4
   sqlite:///D:/SQL/banco.db5
   sqlite:///D:/SQL/banco.db6
Done.


lug_bot,l_bot_small,l_bot_med,l_bot_big
small,1,0,0
med,0,1,0
big,0,0,1


# Banco de dados após todas as transformações:

In [13]:
%%sql

SELECT
    CASE
        WHEN buying = 'vhigh' OR buying = 'high' THEN 'Alto'
        WHEN buying = 'med' OR buying = 'low' THEN 'Baixo'
    END as buying_less,
    CASE
        WHEN safety = 'low' THEN 1
        WHEN safety = 'med' THEN 2
        WHEN safety = 'high' THEN 3
    END as safety_num,
    CASE
        WHEN Target = 'unacc' THEN 1
        WHEN Target = 'acc' THEN 2
        WHEN Target = 'good' THEN 3
        WHEN Target = 'vgood' THEN 4
    END as target_num,
    maint, doors, persons,
    CASE
        WHEN lug_bot = 'small' THEN 1
        ELSE 0
    END AS l_bot_small,
    CASE
        WHEN lug_bot = 'med' THEN 1
        ELSE 0
    END AS l_bot_med,
    CASE
        WHEN lug_bot = 'big' THEN 1
        ELSE 0
    END AS l_bot_big
FROM Car
LIMIT 10;

 * sqlite:///D:/SQL/banco.db2
Done.


buying_less,safety_num,target_num,maint,doors,persons,l_bot_small,l_bot_med,l_bot_big
Alto,1,1,vhigh,2,2,1,0,0
Alto,2,1,vhigh,2,2,1,0,0
Alto,3,1,vhigh,2,2,1,0,0
Alto,1,1,vhigh,2,2,0,1,0
Alto,2,1,vhigh,2,2,0,1,0
Alto,3,1,vhigh,2,2,0,1,0
Alto,1,1,vhigh,2,2,0,0,1
Alto,2,1,vhigh,2,2,0,0,1
Alto,3,1,vhigh,2,2,0,0,1
Alto,1,1,vhigh,2,4,1,0,0


#### Com as variaveis 'buying',lug_bot,safety,Target modificadas**

----------------------------------------------------------------------------------------------------------------------------------------------------------------

# -Binarização

* A binarização é usada para converter a variável categórica em uma representação numérica[0,1],em que normalmente a classe negativa recebe '0' e a classe positiva '1'

**Banco de dados -Dados de paciente com cãncer de mama**

 **Nesse exemplo utilizo um banco de dados disponivel em https://archive.ics.uci.edu/dataset/14/breast+cancer**


-------------------------------------------------------

### Dicionário dos dados

#### Variaveis                    Breast cancer
              
- Class -               ['no-recurrence-events', 'recurrence-events']=[evento não recorente,evento recorrrente]

- Age  -                Idade    

- menopausa  -          Se já está no perido da menopausa

- tamanho_tumor -       Tamanho tumor

- inv_nodes -           Tamanho nodulo

- node_caps   -        Se tem nodulos

- deg_malig-           Grau de malignitude do cancer
- seio-                Qual seio
- quadrante-           Qual quadrante do seio

  -irradiando-           Se o cancer irradiou['yes','no']

In [18]:
df_breast=pd.read_csv('D:/datasets/breast/breast-cancer.csv')
cne = sqlite3.connect('D:/SQL/banco.db8')
cursor=cne.cursor()

In [105]:
df_breast.to_sql ('cancer',cne)

286

In [21]:
%load_ext sql
%sql sqlite:///D:/SQL/banco.db8

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [29]:
%%sql
SELECT * FROM cancer
LIMIT 10


   sqlite:///D:/SQL/banco.db2
   sqlite:///D:/SQL/banco.db6
 * sqlite:///D:/SQL/banco.db8
Done.


index,classe,idade,menopausa,tamanho_tumor,inv_nodes,node_caps,deg_malig,seio,quadrante,irradiando
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
5,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,left,left_low,no
6,no-recurrence-events,50-59,premeno,25-29,0-2,no,2,left,left_low,no
7,no-recurrence-events,60-69,ge40,20-24,0-2,no,1,left,left_low,no
8,no-recurrence-events,40-49,premeno,50-54,0-2,no,2,left,left_low,no
9,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,left_up,no


#### Verifiquei que a variavel 'irradiando' era composta pelas classe ['yes','no'] e binarizei[0,1]

In [24]:
cursor.execute('''
    SELECT DISTINCT irradiando,
    CASE
        WHEN irradiando='no' THEN 0
        WHEN irradiando='yes' THEN 1
    END as classe_irr
    FROM cancer
''')
resultado=cursor.fetchall()
df_breast = pd.DataFrame(resultado, columns=['irradiando', 'classe_irr'])
print(df_breast)

  irradiando  classe_irr
0         no           0
1        yes           1


#### Verifiquei que a variavel 'classe' era composta pelas classe ['no-recurrence-eventss','recurrence-events'] e binarizei -[0,1]

In [25]:
cursor.execute('''
    SELECT DISTINCT classe,
    CASE
        WHEN classe='no-recurrence-events' THEN 0
        WHEN classe='recurrence-events' THEN 1
    END as classe_num
    FROM cancer
''')

resultado=cursor.fetchall()
df_breast = pd.DataFrame(resultado, columns=['classe', 'classe_num'])
print(df_breast)

                 classe  classe_num
0  no-recurrence-events           0
1     recurrence-events           1


# Banco de dados após as transformaçoes

#### Em que nesse banco apenas as colunas 'classe' e 'irradiando' sofreram transformaçoes

In [34]:
%%sql

SELECT 
    
    CASE
        WHEN classe='no-recurrence-events' THEN 0
        WHEN classe='recurrence-events' THEN 1
    END as classe_num,
    idade,
    menopausa,
    tamanho_tumor,
    inv_nodes,
    node_caps,
    deg_malig,
    seio,
    quadrante,
    CASE
        WHEN irradiando='no' THEN 0
        WHEN irradiando='yes' THEN 1
    END as classe_irr
FROM 
    cancer
LIMIT 10;

   sqlite:///D:/SQL/banco.db2
   sqlite:///D:/SQL/banco.db6
 * sqlite:///D:/SQL/banco.db8
Done.


classe_num,idade,menopausa,tamanho_tumor,inv_nodes,node_caps,deg_malig,seio,quadrante,classe_irr
0,30-39,premeno,30-34,0-2,no,3,left,left_low,0
0,40-49,premeno,20-24,0-2,no,2,right,right_up,0
0,40-49,premeno,20-24,0-2,no,2,left,left_low,0
0,60-69,ge40,15-19,0-2,no,2,right,left_up,0
0,40-49,premeno,0-4,0-2,no,2,right,right_low,0
0,60-69,ge40,15-19,0-2,no,2,left,left_low,0
0,50-59,premeno,25-29,0-2,no,2,left,left_low,0
0,60-69,ge40,20-24,0-2,no,1,left,left_low,0
0,40-49,premeno,50-54,0-2,no,2,left,left_low,0
0,40-49,premeno,20-24,0-2,no,2,right,left_up,0


--------------------------------------------


# FIM

